#### Project 3 Submission- Data Analyst Nanodegree - by Nathan Cox
# Open Streetmap Data Wrangling with MongoDB
#### Map Area: Toronto, Ontario

https://mapzen.com/data/metro-extracts/metro/toronto_canada/

## 1. Data Wrangling

### 1.1 Data Acquisition
After downloading the Toronto, Canada dataset from mapzen.com (link above), I extracted the file into the project directory on my machine. In order speed up the investigation step, I made a subset of the file using the following code. It copies every *kth* tag into a new .osm file making the subset file *1/k* times the size of the original file. The code was provided in the Project Details section.

In [1]:
import xml.etree.ElementTree as ET  #library for processing xml

OSM_FILE = "toronto_canada.osm"  #the full .osm Toronto, Canada dataset (input)
SAMPLE_FILE = "toronto_small.osm" #the subset .osm dataset (output)

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')


### 1.2 Data Investigation and Cleaning
Using the reduced dataset, I investigated the following fields to determine the quality of the data and improve it where I could.

#### 1.2.1 Investigating Street Names
To investigate street names, I ran through each element that had a street name tag and inspected the street name. If the street name was not one of the *expected* street names, it was added to a list of unexpected street names. This list was then inspected manually and the following 2 actions were taken:

1. The list of *expected* street names was updated with any valid street names that were found but were not already on the *expected* list.
2. A seperate list was created for street names that were in an incorrect format. This list could then be used to clean the data.

Below is the code used to inspect the data. The 'expected' list is a list of the street names that are in the correct format. The 'mapping' dictionary is a list of street names that need to be corrected. This dictionary is used in the next step to clean the data before entering it into the database.

In [ ]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "toronto_small.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "West", "East", "North", "South", "Way", "Townline", "Terrace",
            "Wood", "Walk", "Vineway", "Sideroad", "Run", "Row", "Ridge", "Starway", "Promenade", "Path", 
            "Pathway", "Point", "Park", "Millway", "Mews", "Meadoway", "Manor", "Line", "Landing", 
            "Hollow", "Hill", "Highway", "Heights", "Grove", "Gate", "Gardens", "Crescent", "Fernway", 
            "Crossing", "Garden", "Common", "Concession", "Close", "Circuit", "Circle" ]

# this variable is compiled by inspecting the program output and adding incorrect street name formats to the dictionary
mapping = {     "St": "Street",
                "St.": "Street",
                "Ave": "Avenue",
                "Ave." : "Avenue",
                "Rd.": "Road",
                "Rd" : "Road",
                "W." : "West",
                "E." : "East",
                "N." : "North",
                "S." : "South",
                "Unionville" : "",
                "Tottenham" : "",
                "Hrbr" : "Harbour",
                "Dr" : "Drive",
                "EHS" : "",
                "Cresent" : "Crescent",
                "By-pass" : "Bypass",
                "Blvd" : "Boulevard",
                "Blvd." : "Boulevard",
                "Alliston" : "" }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    
    osm_file.close()
    print "Audit complete"
    return street_types


def update_name(name, mapping):
    m = street_type_re.search(name) #extracts the street name from the street address string
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(street_type_re, mapping[street_type], name)
            
    return name


def test():
    st_types = audit(OSMFILE)
    pprint.pprint(dict(st_types)) #prints out the list of unexpected street names for visual inspection


if __name__ == '__main__':
    test()

#### 1.2.2 Investigating Postal Codes
To investigate postal codes (Canadian equivalent of zip codes), I ran through each element that had a "addr:postcode" tag and inspected the information. Postal codes are 6 characters long and they alternate a letter followed by a number(eg. A1A 1A1). The most common way to represent a postal code is to capitalize the letters and to have a space after 3 characters. A regular expression was used to determine if the postal code was entered in an expected format (A1A 1A1). The most common exception was if the postal code did not have a space (A1A1A1). Of 684 postal codes in the reduced size data set, approximated 40 had complete information but were missing the space in the middle. 4 other 'postcode' tags did not have correct information or were missing information (less than 6 characters). When the data is entered into the database, the postal codes with the missing space will be corrected to the standard format. Postal codes with missing information will be ignored.

Below is the code used to inspect the data. The regular expressions used to match the 2 postal code formats will be used later to clean the data before loading into the database.


In [ ]:

import xml.etree.cElementTree as ET
import pprint
import re

OSMFILE = "toronto_small.osm"

#expected postal code pattern A1A 1A1
post_code_re = re.compile(r'^[A-Z][0-9][A-Z] [0-9][A-Z][0-9]$') 

#most common incorrect postal code pattern A1A1A1
post_code_re2 = re.compile(r'^[A-Z][0-9][A-Z][0-9][A-Z][0-9]$')

def audit_post_code(post_code_types, post_code):
    m = post_code_re.search(post_code)
    n = post_code_re2.search(post_code)
    
    
    if n:  #matches the most common incorrect pattern (A1A1A1) and corrects by adding a space in the middle
        post_code = re.sub(r'^[A-Z][0-9][A-Z]', post_code[:3] + " ", post_code)
        print post_code      
    elif not m: #postal codes that do not match the expected pattern (A1A 1A1) are added to the list for visual inspection    
        if post_code not in post_code_types:
            post_code_types[post_code] = 1
        else:
            post_code_types[post_code] += 1


def is_post_code(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    post_code_types = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_post_code(tag):
                    audit_post_code(post_code_types, tag.attrib['v'])
    
    osm_file.close()
    return post_code_types


def test():
    post_code_types = audit(OSMFILE)
    pprint.pprint(dict(post_code_types))


if __name__ == '__main__':
    test()

#### 1.2.3 City Names
To investigate city names, I ran through each element that had a "addr:city" tag and inspected the information. There was 1 major formatting exception. A prefix of "City of" or "Town of" was added to many of the city names. 

Below is the code used to inspect the data. A dictionary called "mapping" was created to capture these sections. This dictionary will be used during the data cleaning phase to ensure that data entered into the database is correct.

In [ ]:
import xml.etree.cElementTree as ET
import pprint

OSMFILE = "toronto_small.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


# dictionary that will be used in data cleaning phase
# compiled by visual inspection of output
mapping = {  'City of Brampton' : 'Brampton',
             'City of Burlington' : 'Burlington',
             'City of Hamilton': 'Hamilton',
             'City of Kawartha Lakes': 'Kawartha Lakes',
             'City of Oshawa': 'Oshawa',
             'City of Pickering': 'Pickering',
             'City of St. Catharines': 'St. Catherines',
             'City of Toronto': 'Toronto',
             'City of Vaughan': 'Vaughan',
             'Richmond Hill (Oak Ridges)': 'Richmond Hill',
             'Town of Ajax': 'Ajax',
             'Town of Aurora': 'Aurora',
             'Town of Bradford West Gwillimbury': 'Bradford West Gwillimbury',
             'Town of Caledon': 'Caledon',
             'Town of East Gwillimbury': 'East Gwillimbury',
             'Town of Erin': 'Erin',
             'Town of Grimsby': 'Grimsby',
             'Town of Halton Hills': 'Halton Hills',
             'Town of Innisfil': 'Innisfil',
             'Town of Markham': 'Markham',
             'Town of Milton': 'Milton',
             'Town of Mono': 'Mono',
             'Town of New Tecumseth': 'New Tecumseth',
             'Town of Newmarket': 'Newmarket',
             'Town of Niagara-On-The-Lake': 'Niagara-On-The-Lake',
             'Town of Whitby': 'Whitby',
             'Town of Whitchurch-Stouffville': 'Whitchurch-Stouffville'  }

def audit_city(city_types, city_name):
    if city_name not in city_types:
        city_types[city_name] = 1
    else:
        city_types[city_name] += 1


def is_city(elem):
    return (elem.attrib['k'] == "addr:city")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    city_types = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_city(tag):
                    audit_city(city_types, tag.attrib['v'])
    
    osm_file.close()
    return city_types


def test():
    city_types = audit(OSMFILE)
    pprint.pprint(dict(city_types))


if __name__ == '__main__':
    test()

#### 1.2.4 Cleaning Data and Exporting to json 
To clean the data and get it ready for importing into the database, the .osm file is opened and each element is inspected. If the element is a 'way' or a 'node', that element is then inspected further. Relevant element attributes are stored in Python dictionaries or lists so that they can later be exported to json easily. The attributes that are collected from each element are the type (node or way), latitude, longitude, created date and address (including city name, street name, postal code and other address sub-attributes). 

Before storing the city name, street name and postal code attributes, the text for each attribute is cleaned. For city and street names, the attribute text is looked up in the corresponding 'mapping' dictionary that was created in the investigation phase above. If the 'mapping' dictionary has key matching the attribute text, the corresponding value pair is substituted to clean the data. For postal codes, the most common error is corrected using a regular expression.

The code to clean the osm file and output a json file is below.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json


problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
addr = re.compile(r'(?:addr:).*')

#expected postal code pattern A1A 1A1
post_code_re = re.compile(r'^[A-Z][0-9][A-Z] [0-9][A-Z][0-9]$') 

#most common incorrect postal code pattern A1A1A1
post_code_re2 = re.compile(r'^[A-Z][0-9][A-Z][0-9][A-Z][0-9]$')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

street_mapping = {     "St": "Street",
                "St.": "Street",
                "Ave": "Avenue",
                "Ave." : "Avenue",
                "Rd.": "Road",
                "Rd" : "Road",
                "W." : "West",
                "E." : "East",
                "N." : "North",
                "S." : "South",
                "W" : "West",
                "E" : "East",
                "N" : "North",
                "S" : "South",
                "Unionville" : "",
                "Tottenham" : "",
                "Hrbr" : "Harbour",
                "Dr" : "Drive",
                "EHS" : "",
                "Cresent" : "Crescent",
                "By-pass" : "Bypass",
                "Blvd" : "Boulevard",
                "Blvd." : "Boulevard",
                "Alliston" : "",
                "Adjala" : "",
                "Beeton" : "",
                 "By-pass" : "Bypass"
                 }


city_mapping = {  'City of Brampton' : 'Brampton',
             'City of Burlington' : 'Burlington',
             'City of Hamilton': 'Hamilton',
             'City of Kawartha Lakes': 'Kawartha Lakes',
             'City of Oshawa': 'Oshawa',
             'City of Pickering': 'Pickering',
             'City of St. Catharines': 'St. Catherines',
             'City of Toronto': 'Toronto',
             'City of Vaughan': 'Vaughan',
             'Richmond Hill (Oak Ridges)': 'Richmond Hill',
             'Town of Ajax': 'Ajax',
             'Town of Aurora': 'Aurora',
             'Town of Bradford West Gwillimbury': 'Bradford West Gwillimbury',
             'Town of Caledon': 'Caledon',
             'Town of East Gwillimbury': 'East Gwillimbury',
             'Town of Erin': 'Erin',
             'Town of Grimsby': 'Grimsby',
             'Town of Halton Hills': 'Halton Hills',
             'Town of Innisfil': 'Innisfil',
             'Town of Markham': 'Markham',
             'Town of Milton': 'Milton',
             'Town of Mono': 'Mono',
             'Town of New Tecumseth': 'New Tecumseth',
             'Town of Newmarket': 'Newmarket',
             'Town of Niagara-On-The-Lake': 'Niagara-On-The-Lake',
             'Town of Whitby': 'Whitby',
             'Town of Whitchurch-Stouffville': 'Whitchurch-Stouffville'  }


def clean_post_code(post_code):
    
    #matches the most common incorrect pattern (A1A1A1) and corrects by adding a space in the middle
    if post_code_re.search(post_code):  
        return post_code
              
    #postal codes that do not match the expected pattern (A1A 1A1) are added to the list for visual inspection    
    elif post_code_re2.search(post_code): 
        return re.sub(r'^[A-Z][0-9][A-Z]', post_code[:3] + " ", post_code)

    else:
        return None


def clean_city(city_name):
    if city_name in city_mapping:
        return city_mapping[city_name]
    else:
        return city_name

def clean_streen_names(street_name):
    if street_name in street_mapping:
        return street_mapping[street_name]
    else:
        return street_name

def shape_element(element):
    node = {}
    node['pos']=[]
    
    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        
        if 'lat' in element.attrib:
            node['pos'].append(float(element.attrib['lat']))
            
            if 'lon' in element.attrib:
                node['pos'].append(float(element.attrib['lon']))
        
        
        node['created']={}
            
        for k,v in element.attrib.iteritems():
            if k in CREATED:
                node["created"][k]=v
            elif k=='k':
                pass
            elif k!='lon' and k!='lat':
                node[k]=v
        
        
        node['address']={}
        
        for child in element.iter('tag'):
            k = child.attrib['k']
            v = child.attrib['v']
            
            if not problemchars.match(k):
                if addr.match(k):
                    after_addr = k.partition("addr:")[2]
                    
                    if after_addr == 'postcode':
                        node['address'][after_addr] = clean_post_code(v)
                    
                    elif after_addr == 'city':
                        node['address'][after_addr] = clean_city(v)
                        
                    elif after_addr == 'street':
                        node['address'][after_addr] = clean_streen_names(v)
                    
                    elif ':' not in after_addr:
                        try:
                            node['address'][after_addr] = v
                        except:
                            pass
                            
                else:
                    if ':' in k:
                        a = k.partition(':')[0] 
                        b = k.partition(':')[2]
                        node[a]={}
                        node[a][b] = v
                    elif k!='address':
                        node[k]=v
           
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return 1

def test():
  data = process_map('toronto_small.osm', True)
      

if __name__ == "__main__":
    test()

#### 1.2.5 Loading json File into Database
Once the data is cleaned and in a json format it is easy to load into the database. The following command does it on a system that has mongodb setup.

In [ ]:
mongoimport --db test --collection toronto_small --file /path/to/file/toronto_small.osm.json --jsonArray

## 2. Data Exploration (find patterns and report interesting facts/stats)
Now that the data is cleaned and loaded into the database, it can be explored more easily. The following is a description of some of the data exploration that was done. There is a written explanation as well as mongodb commands and the output from those commands.

### 2.1 Size of toronto_small Collection
The total size of the collection when stored in mongodb is 163,004,032 bytes or 163 MB. The .osm source file and .json intermediate file are 117 MB and 159 MB respectively. One of the reasons the data is larger in mongodb is that the total size that is shown includes the index which adds some additional data. The size of the .osm and .json files were taken from the local file manager application. The following command is how to get the stats of the collection and the output of the command is included. This command was entered into the mongodb prompt.

In [ ]:
> db.toronto_small.stats()
{
    "ns" : "test.toronto_small",
    "count" : 549272,
    "size" : 163004032,
    "avgObjSize" : 296,
    "storageSize" : 174735360,
    "numExtents" : 12,
    "nindexes" : 1,
    "lastExtentSize" : 50798592,
    "paddingFactor" : 1,
    "systemFlags" : 1,
    "userFlags" : 1,
    "totalIndexSize" : 17831856,
    "indexSizes" : {
        "_id_" : 17831856
    },
    "ok" : 1
}


### 2.2 Number of Unique Users
The number of unique users was determined using the following mongodb aggregation pipeline. The output of the following code is:

*{u'_id': None, u'unique_users': 1144}*

In [ ]:
from pymongo import MongoClient
import pprint

def aggregate_unique(collection):
    
    unique_users = collection.aggregate([
        { "$match": { "created.user": { "$exists": True } }},
        { "$group": { "_id": { "user": "$created.user" }, "count": { "$sum": 1 } } },
        { "$group" : { "_id" : None, "unique_users" : { "$sum" : 1 } } }
    ])
    
    return unique_users 

def test():

    client = MongoClient ()
    db = client.test
    collection = db.toronto_small

    unique_users = aggregate_unique(collection)
    
    for a in unique_users:
        pprint.pprint(a)

if __name__ == "__main__":
    test()

### 2.3 Number of Nodes
The number of nodes was determined using the following mongodb aggregation pipeline. The output of the following code is:

{u'_id': None, u'number_of_nodes': 479158}

In [ ]:
#Note - the main function and import statements have been ommitted for this code and all 
#the following aggregations

def count_nodes(collection):
    
    nodes = collection.aggregate([
        { "$match": { "type": "node" } },
        { "$group" : { "_id" : None, "number_of_nodes" : { "$sum" : 1 } } }
    ])
    
    return nodes

### 2.4 Number of Ways
Making a small change to the code from section 2.3 will return the number of ways. The change to the code is:

*{ "$match": { "type": "~~node~~ way" } },*

The result of this code is:

*{u'_id': None, u'number_of_ways': 69720}*

### 2.5 Types and Numbers of Buildings
The aggregation below shows the building types and the number of each type of building in the database.  The output from the aggregation is:

*{u'_id': None, u'count': 539437}  
{u'_id': u'yes', u'count': 5247}  
{u'_id': u'house', u'count': 1177}  
{u'_id': u'residential', u'count': 557}  
{u'_id': u'retail', u'count': 468}  
{u'_id': u'industrial', u'count': 410}  
{u'_id': u'apartments', u'count': 391}  
{u'_id': {u'levels': u'2'}, u'count': 347}  
{u'_id': {u'levels': u'1'}, u'count': 265}  
{u'_id': u'school', u'count': 139}  
{u'_id': {u'material': u'brick'}, u'count': 120}*

Most documents (almost 540,000 out of 550,000) do not have a building tag. The most common building tag is 'yes' which probably means that there is a building. The second most common building tag is 'house' and then 'residential'. This only shows the top 10 most common building tags ($limit is set to 11 because documents without a building tag are also included).

In [ ]:
def count_buildings(collection):
    
    buildings = collection.aggregate([
        { "$group" : { "_id" : "$building", "count" : { "$sum" : 1 } } },
        { "$sort" : { "count" : -1 } },
        { "$limit" : 11 }
    ])
    
    return buildings 

### 2.5 Types and Numbers of Surfaces
The aggregation below shows the surface types and the number of each type of surface in the database.  The output from the aggregation is:

*{u'_id': None, u'count': 533671}  
{u'_id': u'asphalt', u'count': 14235}  
{u'_id': u'paved', u'count': 419}  
{u'_id': u'unpaved', u'count': 333}  
{u'_id': u'concrete', u'count': 286}  
{u'_id': u'gravel', u'count': 85}  
{u'_id': u'dirt', u'count': 79}   
{u'_id': u'wood', u'count': 42}  
{u'_id': u'paving_stones', u'count': 24}  
{u'_id': u'grass', u'count': 21}  
{u'_id': u'ground', u'count': 19}*

In [ ]:
def count_surfaces(collection):
    
    surfaces = collection.aggregate([
        { "$group" : { "_id" : "$surface", "count" : { "$sum" : 1 } } },
        { "$sort" : { "count" : -1 } },
        { "$limit" : 11 }
    ])
    
    return surfaces 

## 3. Suggestions for Improving and Analyzing the Data
This section suggests improvements in the data structure and analysis and discusses the benefits of these improvements.

### 3.1 Improvements
#### 3.1.1. Add an 'Obsolete' Field  
The 'created' field contains the time that an item was added to the dataset. When an item in the physical world is removed (a store closes, a building is demolished, a highway is relocated), instead of removing it from the database, an 'obsolete' field could be added to the item. 
#### 3.1.2 Add a Field for User Feedback
A user feedback field, like upvotes on Reddit, could be used to validate the quality of the data. A user could upvote a map element that is correctly located and the data is correct and downvote an element that is incorrect.
### 3.2 Benefits
#### 3.2.1. Add an 'Obsolete' Field
The benefit of the 'obsolete' field would make it possible to view the map at a specific point in time and view changes in an area over time. 
This could be useful for a city planner or a neighbourhood organization  that is trying to study changes in their neighbourhood to shape policies or regulations.
#### 3.2.2 Add a Field for User Feedback
A user feedback field would allow for very quick data validation. If incorrect data was entered, any user using the map could 'vote' on the validity of the data. A lot of downvotes may indicate suspect data and could be a signal that the data should be verified.